In [ ]:
import torch

if torch.cuda.is_available():
    print("CUDA is available. GPU in use:", torch.cuda.get_device_name(0))
else:
    print("CUDA is NOT available. Using CPU.")

In [ ]:
import psutil
import torch

# RAM
print("RAM Used (GB):", psutil.virtual_memory().used / 1e9)

# VRAM (if using CUDA)
if torch.cuda.is_available():
    print("VRAM Used (GB):", torch.cuda.memory_allocated() / 1e9)

In [ ]:
import torch
torch.cuda.empty_cache()

### Loading the model and transcribing the sample_audio

In [2]:
# c2t model loading example
import whisper_s2t
from whisper_s2t.backends.ctranslate2.model import BEST_ASR_CONFIG


# model_kwargs = {
#     'device': 'cpu',  # Use 'cpu' if you don't have a GPU
#     'compute_type': 'int8', # Note int8 is only supported for CTranslate2 backend, for others only float16 is supported for lower precision.
#     'asr_options': BEST_ASR_CONFIG
# }



ct2_model = whisper_s2t.load_model(
    model_identifier="medium", 
    backend='CTranslate2', 
    compute_type='int8',  # Best for CUDA
    device='cpu',
    asr_options=BEST_ASR_CONFIG,
)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

: 

In [ ]:

# audio_path = "test2.wav"
# files = [audio_path]
# lang_codes = ['ml']
# initial_prompts = [None]
# # Direct Malayalam-to-English translation
# tasks = ['translate']

# ct2_out = ct2_model.transcribe_with_vad(
#     files,
#     lang_codes=lang_codes,
#     tasks=tasks,
#     initial_prompts=initial_prompts,
#     batch_size=16
# )

# # segments = ct2_out[0]  # Get the segments from the first file's output

# # cleaned_segments = [
# #     seg.text for seg in segments
# #     if getattr(seg, "avg_logprob", None) is not None and seg.avg_logprob > -1.0 and getattr(seg, "no_speech_prob", 1.0) < 0.5
# # ]
# # cleaned_transcript = " ".join(cleaned_segments)

# print("c2t English translation:")
# print(ct2_out[0][0])   # English text

In [ ]:
import importlib
import soundfile as sf
import preprocess 
importlib.reload(preprocess)

output_path = "output.wav"
audio_path = "test2.wav"

audio, sr = preprocess.preprocess_audio(
    "test2.wav",
    "output.wav",
)


In [ ]:
lang_codes = ['ml']
initial_prompts = [None]
# Direct Malayalam-to-English translation
tasks = ['translate']


files = ["output.wav"]
    
ct2_out = ct2_model.transcribe_with_vad(
        files,
        lang_codes=lang_codes,
        tasks=tasks,
        initial_prompts=initial_prompts,
        batch_size=16
    )

    # cleaned_segments = [
    #     seg.text for seg in segments
    #     if getattr(seg, "avg_logprob", None) is not None and seg.avg_logprob > -1.0 and getattr(seg, "no_speech_prob", 1.0) < 0.5
    # ]
    # cleaned_transcript = " ".join(cleaned_segments)

print("c2t English translation:")
print(ct2_out)   # English text